## Batch 1 dataset : 299 train + 112 test

In [1]:
!pwd

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training


In [24]:
#the train is in micelle1, michelle1Label
#the test folder is in michelle2, michelle2Label
!pwd

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/michelle1Label


In [66]:
import xml.etree.ElementTree as ET
import os
def read_content(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    list_with_all_boxes = []
    list_with_all_labels = []
    for boxes in root.iter('object'):
        filename = root.find('filename').text
        label = boxes.find('name').text
        ymin, xmin, ymax, xmax = None, None, None, None
        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)
        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_labels.append(label)
        list_with_all_boxes.append(list_with_single_boxes)
    return list_with_all_labels,list_with_all_boxes


In [47]:
#get training labels

gt_train_labels = []
from glob import glob
%cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/michelle1Label
for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/michelle1Label/*.xml'):
    single_image =[]
    image_filename = os.path.basename(file_path)
    #print(image_filename)
    labels,boxes = read_content(image_filename)
    single_image.append(filename)
    single_image.append(labels)
    single_image.append(boxes)
    gt_train_labels.append(single_image)
#print(gt_train_labels[1])
print(len(gt_train_labels))
print(gt_train_labels[1])
%cd ..

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/michelle1Label
299
['39.7565745--104.9899989-135.jpg', ['lamp', 'lamp'], [[40, 269, 74, 496], [568, 274, 633, 462]]]
/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training


In [49]:
#get the list for gt_test_labels
gt_test_labels = []
from glob import glob
%cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/MIchelle2Label
for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/MIchelle2Label/*.xml'):
    single_image =[]
    image_filename = os.path.basename(file_path)
    #print(image_filename)
    labels,boxes = read_content(image_filename)
    single_image.append(filename)
    single_image.append(labels)
    single_image.append(boxes)
    gt_test_labels.append(single_image)
#print(gt_train_labels[1])
print(len(gt_test_labels))
print(gt_test_labels[1])
%cd ..

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/MIchelle2Label
112
['39.7565745--104.9899989-135.jpg', ['sign-h'], [[530, 337, 538, 357]]]
/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training


In [51]:
#convert ground truth image list labels from labm, sign-h, fh to 0,1,2

for i in range(len(gt_train_labels)):
    for index,label in enumerate(gt_train_labels[i][1]):
        if label == 'lamp':
            gt_train_labels[i][1][index] = 0
        if label == 'sign-h':
            gt_train_labels[i][1][index] = 1
        if label == 'fh':
            gt_train_labels[i][1][index] = 2
            
for i in range(len(gt_test_labels)):
    for index,label in enumerate(gt_test_labels[i][1]):
        if label == 'lamp':
            gt_test_labels[i][1][index] = 0
        if label == 'sign-h':
            gt_test_labels[i][1][index] = 1
        if label == 'fh':
            gt_test_labels[i][1][index] = 2

#check example
print(gt_train_labels[2])
print(gt_test_labels[2])

['39.7565745--104.9899989-135.jpg', [0], [[396, 278, 450, 407]]]
['39.7565745--104.9899989-135.jpg', [0], [[433, 283, 457, 450]]]


In [52]:
def sep_class_w_bbox(image_list):
    images_label_0=[]
    images_label_1=[]
    images_label_2=[]
    for single_image in image_list:
        image_label_0 = []  #each image
        image_label_1 = []
        image_label_2 = []  
        bbox_label_0 = []
        bbox_label_1 = []
        bbox_label_2 = []
        label_num = len(single_image[1])
        for i in range(label_num):
            if single_image[1][i] == 0 :
                bbox_label_0.append(single_image[2][i])
            if single_image[1][i] == 1 :
                bbox_label_1.append(single_image[2][i])
            if single_image[1][i] == 2 :
                bbox_label_2.append(single_image[2][i])
        if bbox_label_0 != []:
            image_label_0.append(single_image[0])
            image_label_0.append(bbox_label_0)
        if bbox_label_1 !=[]:
            image_label_1.append(single_image[0])
            image_label_1.append(bbox_label_1)
        if bbox_label_2 !=[]:
            image_label_2.append(single_image[0])
            image_label_2.append(bbox_label_2)
      
        images_label_0.append(image_label_0)
        images_label_1.append(image_label_1)
        images_label_2.append(image_label_2) 

    #remove empty list form the list
    images_label_0 = [ele for ele in images_label_0 if ele != []] 
    images_label_1 = [ele for ele in images_label_1 if ele != []] 
    images_label_2 = [ele for ele in images_label_2 if ele != []]
    return[images_label_0,images_label_1,images_label_2]

In [57]:
print('train class 1 sign-h number is ', len(sep_class_w_bbox(gt_train_labels)[1]))
print('train class 2 fh number is ', len(sep_class_w_bbox(gt_train_labels)[2]))
print('test class 1 sign-h number is ', len(sep_class_w_bbox(gt_test_labels)[1]))
print('test class 2 fh number is ', len(sep_class_w_bbox(gt_test_labels)[2]))

train class 1 sign-h number is  13
train class 2 fh number is  24
test class 1 sign-h number is  5
test class 2 fh number is  14


## Batch 2 dataset: 800 train + 300 test

In [58]:
!pwd

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training


In [68]:
gt_train_labels = []
from glob import glob
%cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img/train
for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img/train/*.xml'):
    single_image =[]
    image_filename = os.path.basename(file_path)
    #print(image_filename)
    labels,boxes = read_content(image_filename)
    single_image.append(filename)
    single_image.append(labels)
    single_image.append(boxes)
    gt_train_labels.append(single_image)

print(len(gt_train_labels))
print(gt_train_labels[1])
%cd ..

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img/train
818
['39.7565745--104.9899989-135.jpg', ['lamp', 'fh'], [[361, 119, 479, 447], [196, 436, 221, 476]]]
/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img


In [72]:
!pwd

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img


In [75]:
#get the list for gt_test_labels
gt_test_labels = []
from glob import glob
%cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img/test/
for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img/test/*.xml'):
    single_image =[]
    image_filename = os.path.basename(file_path)
    #print(image_filename)
    labels,boxes = read_content(image_filename)
    single_image.append(filename)
    single_image.append(labels)
    single_image.append(boxes)
    gt_test_labels.append(single_image)
#print(gt_train_labels[1])
print(len(gt_test_labels))
print(gt_test_labels[1])
%cd ..

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img/test
351
['39.7565745--104.9899989-135.jpg', ['fh', 'lamp'], [[273, 401, 285, 420], [314, 187, 335, 419]]]
/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/training/threeBatch/rmr_img


In [76]:
#convert ground truth image list labels from labm, sign-h, fh to 0,1,2

for i in range(len(gt_train_labels)):
    for index,label in enumerate(gt_train_labels[i][1]):
        if label == 'lamp':
            gt_train_labels[i][1][index] = 0
        if label == 'sign-h':
            gt_train_labels[i][1][index] = 1
        if label == 'fh':
            gt_train_labels[i][1][index] = 2
            
for i in range(len(gt_test_labels)):
    for index,label in enumerate(gt_test_labels[i][1]):
        if label == 'lamp':
            gt_test_labels[i][1][index] = 0
        if label == 'sign-h':
            gt_test_labels[i][1][index] = 1
        if label == 'fh':
            gt_test_labels[i][1][index] = 2

#check example
print(gt_train_labels[2])
print(gt_test_labels[2])

['39.7565745--104.9899989-135.jpg', [0], [[236, 137, 272, 448]]]
['39.7565745--104.9899989-135.jpg', [0], [[305, 314, 320, 409]]]


In [77]:
print('train class 1 sign-h number is ', len(sep_class_w_bbox(gt_train_labels)[1]))
print('train class 2 fh number is ', len(sep_class_w_bbox(gt_train_labels)[2]))
print('test class 1 sign-h number is ', len(sep_class_w_bbox(gt_test_labels)[1]))
print('test class 2 fh number is ', len(sep_class_w_bbox(gt_test_labels)[2]))

train class 1 sign-h number is  24
train class 2 fh number is  96
test class 1 sign-h number is  14
test class 2 fh number is  52
